# Prediksi Total Pengeluaran Berdasarkan Transaksi Pembelian
Proyek Machine Learning untuk submission kelas Dicoding: Machine Learning

**Pendekatan**: Regresi
**Target**: TotalCost

## 1. Pendahuluan
Dalam pengelolaan anggaran perusahaan, penting untuk memahami faktor-faktor yang memengaruhi total pengeluaran dari tiap transaksi. Dengan prediksi yang akurat, perusahaan dapat mengelola anggaran dan negosiasi dengan supplier secara lebih efisien.

## 2. Problem Domain
- **Masalah**: Prediksi total biaya pembelian (`TotalCost`) berdasarkan data transaksi.
- **Solusi**: Model regresi untuk memprediksi nilai `TotalCost`.
- **Manfaat**: Menyediakan insight terhadap pengeluaran berdasarkan jenis barang, supplier, dan pembeli.

## 3. Data Understanding
Kolom-kolom dalam dataset:
- `TransactionID`: ID transaksi
- `ItemName`: Nama barang
- `Category`: Kategori barang
- `Quantity`: Jumlah barang
- `UnitPrice`: Harga satuan
- `TotalCost`: Harga total (target)
- `PurchaseDate`: Tanggal pembelian
- `Supplier`: Pemasok
- `Buyer`: Pembeli

In [ ]:
# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Load dataset
df = pd.read_csv('/content/spend_analysis_dataset.csv')
df.head()

## 4. Data Preparation
- Encode data kategorikal
- Normalisasi fitur numerik
- Split data train/test

In [ ]:
# Encode categorical features
cat_cols = ['Category', 'Supplier', 'Buyer']
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])

In [ ]:
# Feature selection
features = ['Category', 'Quantity', 'UnitPrice', 'Supplier', 'Buyer']
X = df[features]
y = df['TotalCost']

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 5. Modeling
Model: Random Forest Regressor

In [ ]:
# Training model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## 6. Evaluation
Gunakan metrik MAE, RMSE, dan R² Score.

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")

## 7. Kesimpulan
Model regresi mampu memprediksi total pengeluaran dengan cukup akurat.

Langkah selanjutnya:
- Coba model lain (Linear Regression, XGBoost)
- Tuning parameter
- Visualisasi hasil prediksi vs data asli